### 🖋 **Notebook Contents**

0. Initial Setup
1. Business Problem Understanding
2. Data Understanding
3. Data Preparation

****

## `Initial Setup`

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import missingno as msno

import warnings
warnings.filterwarnings("ignore")

In [11]:
data = pd.read_csv("../data/raw/ds_salaries.csv")
print(data.shape)
data.sample(10)

(3755, 11)


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
557,2023,SE,FT,Data Analyst,153600,USD,153600,US,0,US,M
1469,2023,MI,FT,Data Analyst,160000,USD,160000,US,0,US,M
2575,2022,SE,FT,Data Engineer,129300,USD,129300,US,0,US,M
2937,2022,MI,FT,Data Analytics Consultant,113000,USD,113000,US,100,US,L
2660,2022,SE,FT,Data Architect,180000,USD,180000,US,100,US,M
2927,2022,SE,FT,Data Scientist,141525,USD,141525,US,100,US,M
2265,2022,MI,FT,Data Analyst,150000,USD,150000,US,0,US,M
2626,2022,SE,FT,Data Engineer,185000,USD,185000,US,100,US,M
607,2023,SE,FT,Data Scientist,201036,USD,201036,US,0,US,M
3054,2022,SE,FT,Data Scientist,140000,USD,140000,US,100,US,L


In [7]:
display(
    data.info(),
    data.describe(),
    data.describe(include='O')
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


None

,work_year,salary,salary_in_usd,remote_ratio
count,3755.000000,3.755000e+03,3755.000000,3755.000000
mean,2022.373635,1.906956e+05,137570.389880,46.271638
std,0.691448,6.716765e+05,63055.625278,48.589050
min,2020.000000,6.000000e+03,5132.000000,0.000000
25%,2022.000000,1.000000e+05,95000.000000,0.000000
50%,2022.000000,1.380000e+05,135000.000000,0.000000
75%,2023.000000,1.800000e+05,175000.000000,100.000000
max,2023.000000,3.040000e+07,450000.000000,100.000000


,experience_level,employment_type,job_title,salary_currency,employee_residence,company_location,company_size
count,3755,3755,3755,3755,3755,3755,3755
unique,4,4,93,20,78,72,3
top,SE,FT,Data Engineer,USD,US,US,M
freq,2516,3718,1040,3224,3004,3040,3153
